### 16/11/23

This notebook is intended to use for working with granule IDs of S1 granules and mapping them and exporting as SHP

GEE is used for this job

Can also include other GEE testing and experimenting

###### home_town


In [1]:
import ee
import IPython.display as disp
import folium
import random
import json
import fiona
from shapely.geometry import Polygon
from shapely.geometry import mapping
import pandas as pd

#### ANNAÐ TESTING 

In [ ]:
# Load the dataset

import geemap
dataset = ee.FeatureCollection('WCMC/WDPA/current/polygons')

# # Define visualization parameters
visParams = {
  'palette': ['2ed033', '5aff05', '67b9ff', '5844ff', '0a7618', '2c05ff'],
  'min': 0.0,
  'max': 1550000.0,
  'opacity': 0.8,
}

# # Create an image and paint the dataset on it
image = ee.Image().float().paint(dataset, 'REP_AREA')

map_l8 = geemap.Map(center=[37.5010, -122.1899], zoom=10)

map_l8.add_layer(image, visParams, 'false color composite')

In [7]:
import ee

# Authenticate to the Earth Engine servers


# Load the WDPA dataset
dataset = ee.FeatureCollection('WCMC/WDPA/current/polygons')

# Specify the value you want to filter on (replace with your specific WDPAID)
wdpa_id_to_filter = 'Vindelfjallen'

# Filter the FeatureCollection based on the WDPAID property
filtered_collection = dataset.filter(ee.Filter.eq('ORIG_NAME', wdpa_id_to_filter))

# Print the filtered collection
print("Filtered Collection:", filtered_collection.getInfo())

# Map the filtered collection to visualize it on the map
# Note: Visualization in Python requires using other libraries like folium or ipyleaflet
# You can adapt the code accordingly based on your preferred visualization library


Filtered Collection: {'type': 'FeatureCollection', 'columns': {'CONS_OBJ': 'String', 'DESIG': 'String', 'DESIG_ENG': 'String', 'DESIG_TYPE': 'String', 'GIS_AREA': 'Float', 'GIS_M_AREA': 'Float', 'GOV_TYPE': 'String', 'INT_CRIT': 'String', 'ISO3': 'String', 'IUCN_CAT': 'String', 'MANG_AUTH': 'String', 'MANG_PLAN': 'String', 'MARINE': 'String', 'METADATAID': 'Integer', 'NAME': 'String', 'NO_TAKE': 'String', 'NO_TK_AREA': 'Float', 'ORIG_NAME': 'String', 'OWN_TYPE': 'String', 'PARENT_ISO': 'String', 'PA_DEF': 'String', 'REP_AREA': 'Float', 'REP_M_AREA': 'Float', 'STATUS': 'String', 'STATUS_YR': 'Integer', 'SUB_LOC': 'String', 'SUPP_INFO': 'String', 'VERIF': 'String', 'WDPAID': 'Float', 'WDPA_PID': 'String', 'system:index': 'String'}, 'version': 1706712542961615, 'id': 'WCMC/WDPA/current/polygons', 'properties': {'system:time_start': 1704067200000, 'system:time_end': 1706745600000, 'system:asset_size': 2051514564}, 'features': []}


#### --END ANNAÐ TESTING 

In [2]:
#funcy functions
def unique_lister(listi):
    #unique basic list
    unique_list = []
    for item in listi:
        if item not in unique_list:
            unique_list.append(item)
    return unique_list

def unique_lister_ee(listi):
    #unique list for ee objects
    unique_list = []
    for item in listi:
        if item not in unique_list:
            unique_list.append(item)
    return unique_list
# this func flips the coordinates to follow (lon, lat) convention
def flip_coords(footprint_str):
    coordinate_pairs = []
    for pair_str in footprint_str:
        temp = []
        temp = [round(float(pair_str[1]),1), round(float(pair_str[0]),1)]
        coordinate_pairs.append(
            temp
        )  # Swap lat and lon to follow (lon, lat) convention
    pairs_within_pairs = [coordinate_pairs]
    return pairs_within_pairs

In [2]:
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


In [5]:
import os

# workspace_directory = r'C:\Users\trygg\Documents\Master_project' #HOME
workspace_directory = r'C:\Users\tryggvisi\Documents\my-awesome-masters-project' #WORK
# Change the current working directory to the specified workspace
os.chdir(workspace_directory)

In [4]:
# dummy region
# Vindefjallen
# geoJSON = {
#     "type": "FeatureCollection",
#     "features": [
#         {
#             "type": "Feature",
#             "properties": {},
#             "geometry": {
#                 "coordinates": [
#                     [
#                         [17.82718737874535, 65.6683016285551],
#                         [15.541817295249245, 66.40781169335355],
#                         [14.558470805139848, 66.05183130712231],
#                         [17.23535847266129, 65.14922801911618],
#                         [17.82718737874535, 65.6683016285551],
#                     ]
#                 ],
#                 "type": "Polygon",
#             },
#         }
#     ],
# }
# coords = geoJSON['features'][0]['geometry']['coordinates']
# aoi = ee.Geometry.Polygon(coords)

In [6]:
#AOI fyrir Vindefjallen study area
# Define the path to the GeoJSON file
# geojson_path = r"C:\Users\trygg\Documents\Master_project\Testing\Vindefjallen_testing\Arc_Vindefjellen\temp\Vindefjallen_e_FeaturesToJSO.geojson"
geojson_path = r"Testing\Vindefjallen_testing\Arc_Vindefjellen\temp\Vindefjallen_e_FeaturesToJSO.geojson"


with open(geojson_path) as f:
    data = json.load(f)
aoi = ee.Geometry.Polygon(data['features'][0]['geometry']['coordinates'])

In [7]:
#information about the granule
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2022-01-01'),ee.Date('2022-02-28')) 
                       .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                       .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                       .first() 
                       .clip(aoi))
ffa_db.bandNames().getInfo()
#get orbit number   
orbitproperties = ffa_db.get('orbitProperties_pass').getInfo()
relativeOrbitNumber = ffa_db.get('relativeOrbitNumber_start').getInfo()
#print both
print(orbitproperties)
print(relativeOrbitNumber)

DESCENDING
66


In [ ]:
# get multitemporal data  (S1)
date_format = 'YYYY-MM-dd'
# date_format = 'dd/MM/YYYY'
date_1 = ee.Date('2017-01-01')
date_2 = ee.Date('2022-04-01')
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD')
           .filterBounds(aoi)
           .filterDate(date_1,date_2)
         #   .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
           #.filter(ee.Filter.eq('relativeOrbitNumber_start', relativeOrbitNumber))
           .filter(ee.Filter.eq('instrumentMode', 'IW'))
           .map(lambda img: img.set('date', ee.Date(img.date()).format(date_format)))
           .sort('date'))

timestamplist = (im_coll.aggregate_array('date')
                #  .map(lambda d: ee.String('').cat(ee.String(d)))
                 .getInfo())
print(len(timestamplist))
for i in timestamplist:
    print(i)

In [9]:
#converts to a ee list
im_list = im_coll.toList(im_coll.size()) 
#converts to a traditional list
s1_coords_listi = im_list.getInfo() 

#### Date filtering

In [ ]:
#ná í dates frá excel skrá
file_path = r"Testing\Vindefjallen_testing\data\vindefjallen_dates.txt" #file með unique dates frá excel skrá um vindefjallen

# Open the file in read mode
with open(file_path, 'r') as file:
    # Read lines from the file into a list, removing newline characters
    vindefjallen_dates = [line.strip() for line in file.readlines()]

# Print the list of dates(
vindefjallen_dates

In [12]:
#hérna er náð í öll dates sem eru í s1_coords_listi sem er listi sem nær yfir alla granules sem passa við aoi
# Extract 'date' values from 'properties' dictionary in s1_coords_listi
s1_dates = [item['properties']['date'] for item in s1_coords_listi]

# Check if each date in s1_dates is in vindefjallen_dates
filtered_s1_coords_listi = [item for item in s1_coords_listi if item['properties']['date'] in vindefjallen_dates]

# Print the filtered list
print(len(filtered_s1_coords_listi))


85


#### Folium mapping

In [25]:
#print the footprint coordinates
for i in range(len(s1_coords_listi)):
    print(i, flip_coords(s1_coords_listi[i]['properties']['system:footprint']['coordinates']))


0 [[[68.0, 21.0], [68.0, 21.0], [67.9, 20.6], [67.8, 19.2], [67.7, 17.5], [67.5, 15.3], [67.5, 15.2], [67.3, 15.3], [67.1, 15.4], [66.8, 15.6], [66.5, 15.8], [66.2, 16.0], [66.0, 16.2], [65.8, 16.3], [66.0, 18.1], [66.2, 20.0], [66.4, 21.7], [66.4, 21.7], [66.4, 21.7], [67.9, 21.1], [68.0, 21.0]]]
1 [[[66.7, 16.4], [66.7, 16.4], [66.9, 13.6], [67.1, 10.8], [67.1, 10.8], [67.1, 10.8], [66.7, 10.7], [66.4, 10.5], [66.0, 10.4], [65.6, 10.2], [65.6, 10.2], [65.5, 12.2], [65.4, 13.8], [65.2, 15.5], [65.2, 15.5], [65.2, 15.5], [65.2, 15.6], [65.5, 15.7], [66.5, 16.4], [66.6, 16.4], [66.7, 16.4]]]
2 [[[65.0, 14.8], [64.7, 17.5], [64.8, 17.5], [64.9, 17.6], [65.0, 17.7], [65.4, 17.9], [65.7, 18.1], [66.1, 18.3], [66.1, 18.3], [66.1, 18.4], [66.2, 18.4], [66.2, 18.4], [66.3, 17.4], [66.7, 12.7], [66.7, 12.7], [66.6, 12.7], [66.1, 12.4], [65.3, 12.2], [65.2, 12.1], [65.2, 12.1], [65.2, 12.5], [65.0, 14.8]]]
3 [[[64.4, 15.8], [64.4, 16.3], [64.4, 16.3], [64.5, 16.2], [64.6, 16.2], [65.6, 15.8], [

In [ ]:
foot_dict = {}
print("Granule footprints:")
for key in range(len(s1_coords_listi)):
    
    foot_dict[key] = {"footprint": flip_coords(s1_coords_listi[key]['properties']['system:footprint']['coordinates']), 
                      "granule_id": s1_coords_listi[key]['properties']['system:index'],
                      "date": s1_coords_listi[key]['properties']['date']}
foot_dict

In [27]:
# Create a Folium map centered on a location
m = folium.Map(location=[66.0, 23.0], zoom_start=5)
# Add the footprints to the map from the dictionary with footprints and apply random colors
for key, coordinates_list in foot_dict.items():
    for coordinates in coordinates_list:
        # Generate a random color in hexadecimal format
        random_color = "#{:02x}{:02x}{:02x}".format(
            random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
        )
        folium.Polygon(
            locations=foot_dict[key]['footprint'],
            color=random_color,
            fill=True,
            fill_color=random_color,
            fill_opacity=0.2,
            popup=folium.Popup(f'GranuleID: {foot_dict[key]["granule_id"]}\nDate: {foot_dict[key]["date"]}', parse_html=True),
            highlight_function= lambda feat: {'fillColor': 'blue'}


        ).add_to(m)
folium.Polygon(
    locations=[(lat, lon) for lon, lat in coords[0]],
    color='white',
    fill=True,
    fill_color='white',
    fill_opacity=0.5,
    popup=folium.Popup("<b>Green Polygon</b><br>Additional Info", parse_html=False),

).add_to(m)
# m.save("coordinates_map2.html")

m

NameError: name 'coords' is not defined

#### Polygon shapefile maker

In [ ]:
#print the footprint coordinates
for i in range(len(filtered_s1_coords_listi)):
    print(i, filtered_s1_coords_listi[i]['properties']['system:footprint']['coordinates'])
    #print granule id
    print(filtered_s1_coords_listi[i]['properties']['system:index'])
    print(filtered_s1_coords_listi[i]['properties']['date'])
    print("\n")


In [ ]:
# s1_coords_listi[0]['properties']['system:index'] #Til að fá granule id
# s1_coords_listi[0]['properties']['date'] #til að fá date
instrumentMode = s1_coords_listi[0]['properties']['instrumentMode']
print(instrumentMode)
orbitProperties_pass = s1_coords_listi[0]['properties']['orbitProperties_pass']
print(orbitProperties_pass)
relativeOrbitNumber_stop = s1_coords_listi[0]['properties']['relativeOrbitNumber_stop']
print(relativeOrbitNumber_stop)
transmitterReceiverPolarisation = ','. join(s1_coords_listi[0]['properties']['transmitterReceiverPolarisation'])
print(transmitterReceiverPolarisation)

In [ ]:
#This cell is designed to take the dictionary foot_dict and convert it to a shapefile. where each granule is a polygon

shapefile_listi = filtered_s1_coords_listi


# Define the output shapefile path and schema
output_shapefile = "outputs/granules_2017-2022_vindefjellen.shp"
schema = {
    'geometry': 'Polygon',
    'properties': {'id': 'int', 
                   'name': 'str', 
                   'date': 'str',
                   'dateStart': 'str',
                   'instrMode': 'str', 
                   'orbitProp': 'str', 
                   'OrbitNr': 'str', 
                   'Pols': 'str'}
    }

# Define the CRS for WGS84
crs_wgs84 = {'init': 'epsg:4326'}  # 'epsg:4326' is the code for WGS84

# Create the shapefile
with fiona.open(output_shapefile, 'w', 'ESRI Shapefile', schema, crs=crs_wgs84) as output:
    # Create a polygon using the provided coordinates

    for i in range(len(shapefile_listi)): #looped through shapefile_listi which has a list of all identified granules
        print(i)
        coordinates = [shapefile_listi[i]['properties']['system:footprint']['coordinates']]
        polygon = Polygon(coordinates[0])
        #converts list to string
        transmitterReceiverPolarisation = ','. join(shapefile_listi[i]['properties']['transmitterReceiverPolarisation'])
        # Create a datetime object
        date_object = datetime.utcfromtimestamp(shapefile_listi[i]['properties']['segmentStartTime'] / 1000 )
        # Format the date as a string
        formatted_date = date_object.strftime('%Y-%m-%d %H:%M:%S UTC')
    # Define the properties for the polygon
        props = {'id': i,
                 'name': shapefile_listi[i]['properties']['system:index'],
                 'date': shapefile_listi[i]['properties']['date'], 
                 'dateStart': formatted_date,
                 'instrMode': shapefile_listi[i]['properties']['instrumentMode'], 
                 'orbitProp': shapefile_listi[i]['properties']['orbitProperties_pass'], 
                 'OrbitNr': shapefile_listi[i]['properties']['relativeOrbitNumber_stop'], 
                 'Pols': transmitterReceiverPolarisation,
                 }

    # Write the feature to the shapefile
        output.write({
            'geometry': mapping(polygon),
            'properties': props,
        })


#### Code for data managements

- add coords to each measurement. import each sheet into R. Add x & y coords when the Triangle lines match

- import the doc to python. Convert the coords to wgs84.
 
- find an algorithm for checking if a given point will be inside of the variably sized footprint areas

- check every individual corner for what granule fits inside it, and add it to a ; seperate list, or just list

- or, iterate through each granule footprint, check every measurement, and add it if it's a match

In [14]:
def point_in_polygon(x, y, polygon):
    """
    Check if a point (x, y) is inside a polygon defined by a list of coordinates.
    
    Parameters:
    - x, y: Coordinates of the point to be checked.
    - polygon: List of tuples representing the vertices of the polygon.

    Returns:
    - True if the point is inside the polygon, False otherwise.
    """
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y) and y <= max(p1y, p2y) and x <= max(p1x, p2x):
            if p1y != p2y:
                xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                if p1x == p2x or x <= xinters:
                    inside = not inside
        p1x, p1y = p2x, p2y

    return inside

# Example usage:
# polygon = [(0, 0), (0, 5), (5, 5), (5, 0), (0, 0)]
polygon = filtered_s1_coords_listi[0]['properties']['system:footprint']['coordinates']
point = (15.6850470, 65.8904299) #out
# point = (15.6698234, 65.8602970) #in
# point = (13.3915035, 64.6581848)
point = (16.0660112, 65.9290453)

if point_in_polygon(point[0], point[1], polygon):
    print(f"The point {point} is inside the polygon.")
else:
    print(f"The point {point} is outside the polygon.")


The point (16.0660112, 65.9290453) is outside the polygon.


In [15]:
#import the excel file with Vindefjallen corner coordinates and measurements.

# Specify the file path to your Excel CSV file
csv_file_path = r"Testing\Vindefjallen_testing\data\vindefjallen_corner_coords.csv"

# Read the CSV file into a DataFrame
df_measurements = pd.read_csv(csv_file_path)
#Remove all nan values from the dataframe
df_measurements_filter = df_measurements.dropna(subset=['SnowDepth'])

# Convert the DataFrame to a NumPy array for easier coding
vindefjallen_triangleCorners_numpy = df_measurements_filter.to_numpy()

# Print the first few rows of the DataFrame
print(df_measurements.head())

#Code from some tutorial of pandas dataframe
# for i in range(len(df_measurements_filter)):
#     print(df_measurements_filter.iloc[i]['ID'], df_measurements_filter.iloc[i]['SnowDepth'])

   ID Triangle.x Corner.x  CensusDate SnowDepth TriangleCorner  \
0   1      AC-42        A  2017-02-21        50         AC-42A   
1   2      AC-38        A  2017-02-26        50         AC-38A   
2   3      AC-37        A  2017-02-26        45         AC-37A   
3   4      AC-31        A  2017-02-26        20         AC-31A   
4   5      AC-39        A  2017-03-04        60         AC-39A   

   xcoord_EPSG_4326  ycoord_EPSG_4326  
0         15.760981         65.908407  
1         16.028612         65.896595  
2         15.848811         65.938003  
3         15.595766         65.984498  
4         15.936407         65.881455  


In [68]:
#Test hvort að það virki að finna granule id fyrir hvern punkt sem er í excel skránni
númer = 0
for i in vindefjallen_triangleCorners_numpy:
    # print(i)
    if i[5] == 'AC-41C':
        print(i)
        númer = i[0]- 1
        break
print(númer)
vindefjallen_triangleCorners_numpy[númer][6], vindefjallen_triangleCorners_numpy[númer][7]

[305 'AC-41' 'C' '2017-03-04' '125' 'AC-41C' 16.3075941203887
 65.8047825499351]
304


(16.3075941203887, 65.8047825499351)

In [121]:
#hver mæling úr excel skránni er testuð hvort að hún sé innan granule og búin að dictionary sem heldur utan um það
ordabok_granule_id = {}
for i in range(len(filtered_s1_coords_listi)):
    polygon = filtered_s1_coords_listi[i]['properties']['system:footprint']['coordinates']
    # print(polygon)
    measurment_in_granule_ID = []
    measurment_in_granule_cornerTriangle = []
    measurment_in_granule_date = []
    measurment_in_granule_depth = []

    for j in range(len(vindefjallen_triangleCorners_numpy)):
        point = (vindefjallen_triangleCorners_numpy[j][6], vindefjallen_triangleCorners_numpy[j][7])
        
        if point_in_polygon(point[0], point[1], polygon):
            # print(f"The point {vindefjallen_triangleCorners_numpy[j][5]} ({point}) is inside the polygon.")
            #check if dates are the same
            if vindefjallen_triangleCorners_numpy[j][3] == filtered_s1_coords_listi[i]['properties']['date']:
                measurment_in_granule_cornerTriangle.append(vindefjallen_triangleCorners_numpy[j][5])
                measurment_in_granule_date.append(vindefjallen_triangleCorners_numpy[j][3])
                measurment_in_granule_depth.append(vindefjallen_triangleCorners_numpy[j][4])
                measurment_in_granule_ID.append(vindefjallen_triangleCorners_numpy[j][0])
    #     else:
    #         print(f"The point {vindefjallen_triangleCorners_numpy[j][5]} ({point}) is outside the polygon.")

    # print('-----------------------------------')
    if len(measurment_in_granule_date) == 0:
        measurment_in_granule_date.append('')

    ordabok_granule_id[i] = {'granule_ID': filtered_s1_coords_listi[i]['properties']['system:index'], 
                             'orbitProp': filtered_s1_coords_listi[i]['properties']['orbitProperties_pass'],
                             'orbitNr': filtered_s1_coords_listi[i]['properties']['relativeOrbitNumber_stop'],
                             'date': filtered_s1_coords_listi[i]['properties']['date'],
                             'measurement_id': measurment_in_granule_ID,
                             'measurement_cornerTriangle': measurment_in_granule_cornerTriangle,
                             'measurement_date': measurment_in_granule_date[0],
                             'measurement_depth': measurment_in_granule_depth,
                             'nr_of_measurements': len(measurment_in_granule_ID)
                                }
    # print(ordabok_granule_id)


In [18]:
#Filterað burt alla kernels sem hafa ekki neina mælingu
# Create a list of keys to be removed
keys_to_remove = [key for key, value in ordabok_granule_id.items() if value.get('measurement_date', '') == '']

# Remove items from the original dictionary
for key in keys_to_remove:
    del ordabok_granule_id[key]

print(len(ordabok_granule_id))

45


In [22]:
#Export the dictionary to excel file
df_granules = pd.DataFrame.from_dict(ordabok_granule_id, orient='index')
loc = r"Testing\Vindefjallen_testing\data\vindefjallen_granules.xlsx"
df_granules.to_excel(loc, index_label='Item')

In [19]:
ordabok_granule_id

{1: {'granule_ID': 'S1B_IW_GRDH_1SDV_20170221T163839_20170221T163904_004401_007A7F_051D',
  'orbitProp': 'ASCENDING',
  'orbitNr': 175,
  'date': '2017-02-21',
  'measurement_id': [299],
  'measurement_cornerTriangle': ['AC-42C'],
  'measurement_date': '2017-02-21',
  'measurement_depth': ['110'],
  'nr_of_measurements': 1},
 2: {'granule_ID': 'S1B_IW_GRDH_1SDV_20170226T052839_20170226T052904_004467_007C67_EAFC',
  'orbitProp': 'DESCENDING',
  'orbitNr': 66,
  'date': '2017-02-26',
  'measurement_id': [2, 3, 4, 151, 152, 153, 300, 301, 302],
  'measurement_cornerTriangle': ['AC-38A',
   'AC-37A',
   'AC-31A',
   'AC-38B',
   'AC-37B',
   'AC-31B',
   'AC-38C',
   'AC-37C',
   'AC-31C'],
  'measurement_date': '2017-02-26',
  'measurement_depth': ['50', '45', '20', '75', '70', '60', '75', '20', '65'],
  'nr_of_measurements': 9},
 3: {'granule_ID': 'S1A_IW_GRDH_1SDV_20170304T052932_20170304T052957_015538_01988A_BE5D',
  'orbitProp': 'DESCENDING',
  'orbitNr': 66,
  'date': '2017-03-04',
 

In [55]:
#every unique TriangleCorner in the excel file
unique_TriangleCorner_list = []
for i in vindefjallen_triangleCorners_numpy:
    if i[5] not in unique_TriangleCorner_list:
        unique_TriangleCorner_list.append(i[5])
unique_TriangleCorner_list.sort()
print(unique_TriangleCorner_list)
print(len(unique_TriangleCorner_list))

['AC-21A', 'AC-21B', 'AC-21C', 'AC-22A', 'AC-22B', 'AC-22C', 'AC-23A', 'AC-23B', 'AC-23C', 'AC-24A', 'AC-24B', 'AC-24C', 'AC-25A', 'AC-25B', 'AC-25C', 'AC-26A', 'AC-26B', 'AC-26C', 'AC-27A', 'AC-27B', 'AC-27C', 'AC-28A', 'AC-28B', 'AC-28C', 'AC-29A', 'AC-29B', 'AC-29C', 'AC-30A', 'AC-30B', 'AC-30C', 'AC-31A', 'AC-31B', 'AC-31C', 'AC-32A', 'AC-32B', 'AC-32C', 'AC-33A', 'AC-33B', 'AC-33C', 'AC-34A', 'AC-34B', 'AC-34C', 'AC-35A', 'AC-35B', 'AC-35C', 'AC-36A', 'AC-36B', 'AC-36C', 'AC-37A', 'AC-37B', 'AC-37C', 'AC-38A', 'AC-38B', 'AC-38C', 'AC-39A', 'AC-39B', 'AC-39C', 'AC-40A', 'AC-40B', 'AC-40C', 'AC-41A', 'AC-41B', 'AC-41C', 'AC-42A', 'AC-42B', 'AC-42C', 'AC-43A', 'AC-43B', 'AC-43C', 'AC-46A', 'AC-46B', 'AC-46C']
72


In [181]:
#Finding the granules that have a measurement in a specific triangle corner
keyList = list(ordabok_granule_id.keys()) #list of all keys in the dictionary
def lookingForATriangleMatch(triangleCorner):
    granue_id = [] #list of granule id's that have a measurement in the triangle corner
    date = []
    depth = []
    measurement_id = []
    measurements_count = []
    for i in keyList:#loop through all granules
        if triangleCorner in ordabok_granule_id[i]['measurement_cornerTriangle']:
            # print(ordabok_granule_id[i]['granule_ID'], ordabok_granule_id[i]['measurement_date'], ordabok_granule_id[i]['measurement_depth'], ordabok_granule_id[i]['measurement_id'], ordabok_granule_id[i]['nr_of_measurements'])
            # print("--------------------------------------------------")
            granue_id.append(ordabok_granule_id[i]['granule_ID'])
            date.append(ordabok_granule_id[i]['measurement_date'])
            # depth.append(ordabok_granule_id[i]['measurement_depth'])
            # measurement_id.append(ordabok_granule_id[i]['measurement_id'])
            # measurements_count.append(ordabok_granule_id[i]['nr_of_measurements'])
    return granue_id, date
# , depth, measurement_id, measurements_count


lookingForATriangleMatch('AC-42A')

(['S1A_IW_GRDH_1SDV_20190311T053749_20190311T053814_026286_02F014_21C9',
  'S1A_IW_GRDH_1SDV_20210316T162310_20210316T162335_037026_045B8A_ADD8',
  'S1A_IW_GRDH_1SDV_20220316T163127_20220316T163152_042349_050C83_C328'],
 ['2019-03-11', '2021-03-16', '2022-03-16'])

In [171]:
def depth_matching_with_date(triangleCorner, date):
    snow_depth = []
    measurement_id = []
    for i in vindefjallen_triangleCorners_numpy:
        for idate in date:
            if triangleCorner == i[5] and idate == i[3]:
                # print(i[0], i[1], i[2], i[3], i[4], i[5], i[6], i[7])
                snow_depth.append(i[4])
                measurement_id.append(i[0])
    return snow_depth, measurement_id


deits = ['2017-03-17', '2018-03-30', '2018-03-30', '2019-04-08', '2020-04-28', '2021-04-12', '2022-03-14']


depth_matching_with_date('AC-21A', deits)

(['60', '85', '85', '85', '200', '60', '155'], [20, 30, 30, 74, 90, 103, 138])

In [131]:
#looking for a triangle match and return the coordinates of the triangle corner
def lookingForTriangleCoords(triangleCorner):
    x_coord = 0
    y_coord = 0
    for j in vindefjallen_triangleCorners_numpy:
        if triangleCorner == j[5]:
            # print(j[5], j[6], j[7])
            x_coord = j[7]
            y_coord = j[6]
            break
    return x_coord, y_coord
lookingForTriangleCoords('AC-40A')       

(65.8722797340435, 16.0897624785653)

In [184]:
#búa til ultimate point shape file
# unique_TriangleCorner_list
ordabok_points_granules = {}
for i in unique_TriangleCorner_list:
    # print(i)
    x_coord, y_coord = lookingForTriangleCoords(i)
    granule_id, date = lookingForATriangleMatch(i)
    # print(date)
    snow_depth = []
    # for idate in date:
    snow_depth, measurement_id = depth_matching_with_date(i, date)
        #
    # snow_depth.append(depth_matching_with_date(i, date))
        # print(i, j)
        # snow_depth.append(snow_depth_function)
    # print(snow_depth)
    # print('XXX')
    # print(x_coord, y_coord)
    # print(granule_id)
    # print(date)
    # print("--------------------------------------------------")
    #build a dictionary
    ordabok_points_granules[i] = {
                            'granule_ID': granule_id, 
                            'date': date,
                            'snow_depth': snow_depth,
                            'measurement_id': measurement_id,
                            'nr_of_measurements': len(snow_depth),
                            'x_coord': x_coord,
                            'y_coord': y_coord
                            }

In [186]:
#export the dictionary to excel file
df_points = pd.DataFrame.from_dict(ordabok_points_granules, orient='index')
loc = r"Testing\Vindefjallen_testing\data\points_with_granuesDepthXY2.xlsx"
df_points.to_excel(loc, index_label='Item')
